In [1]:
from IPython.core.debugger import set_trace

#%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="dark_background")

Import Dataset

In [2]:
txt_data = pd.read_csv('..\Data\Sentiment Analysis Dataset.csv' , sep='|', names=['col1'])

#split data into columns with ','
txt_data = txt_data.col1.str.split(',',  3, expand=True)
txt_data.columns = list(txt_data.iloc[0])
txt_data = txt_data.drop(0)
txt_data.index = np.subtract(txt_data.index, 1)
# x = txt_data.groupby('Sentiment')
# l=[x.get_group(i)['SentimentText'] for i in x.groups]

In [3]:
dataset = pd.concat([txt_data['SentimentText'], txt_data['Sentiment']], axis = 1)
dataset.columns = ['text', 'target']

In [4]:
dataset

,text,target
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0
...,...,...
1577833,Zzzzzz.... Finally! Night tweeters!,1
1577834,"""Zzzzzzz, sleep well people """,1
1577835,ZzzZzZzzzZ... wait no I have homework.,0
1577836,"""ZzZzzzZZZZzzz meh, what am I doing up again? """,0


In [27]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('../Data Preprocesed')
from data_preprocess import *

import pickle

In [29]:
text = NLP_preprocess(dataset)
text.preprocess_data()

text.set_tokenizer()
import pickle

# saving
with open('tokenizer_1M.pickle', 'wb') as handle:
    pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
train_dataset = dataset[:1000000]
train_labels = train_dataset.target.astype(float)
train_padded = text.tokenize_and_pad(train_dataset, train = True)
print(train_padded)

[[    56 196012    170 ...      0      0      0]
 [   155     34    796 ...      0      0      0]
 [   211    105   3687 ...      0      0      0]
 ...
 [   174    651    265 ...      0      0      0]
 [   174     36   5425 ...      0      0      0]
 [   174   5067      7 ...      0      0      0]]


In [8]:
test_dataset = dataset[:1000000]
test_labels = test_dataset.target.astype(float)
test_padded = text.tokenize_and_pad(test_dataset, train = False)
print(test_padded)

[[    56 196012    170 ...      0      0      0]
 [   155     34    796 ...      0      0      0]
 [   211    105   3687 ...      0      0      0]
 ...
 [   174    651    265 ...      0      0      0]
 [   174     36   5425 ...      0      0      0]
 [   174   5067      7 ...      0      0      0]]


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Embedding(text.num_words, 8, input_length=text.max_length))
model.add(LSTM(16, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))

optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             6266224   
_________________________________________________________________
lstm (LSTM)                  (None, 16)                1600      
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 6,267,841
Trainable params: 6,267,841
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    train_padded, train_labels, epochs=20, validation_data=(test_padded, test_labels),
)

In [22]:
import random
#test_neg = random.choice(neg.values)
i = random.choice(test_dataset.index)
if test_dataset.loc[i]['target'] == '0':
    print('negative: ', test_dataset.loc[i]['text'])
else:
    print('positive: ', test_dataset.loc[i]['text'])
test_sequences = text.tokenizer.texts_to_sequences([test_dataset.loc[i]['text']])
test_padded2 = pad_sequences(
    test_sequences, maxlen=text.max_length, padding="post", truncating="post"
)
model.predict(test_padded2)

positive:  niccisnail ill definitely write look forward article re5 write fun


array([[0.9780675]], dtype=float32)